# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [1]:
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import edit_distance
from nltk.stem.porter import PorterStemmer
import nltk
import string
from nltk.collocations import *

In [2]:
lines = open("holbrook.txt").readlines()                            # read the txt file into lines
data = []                                                           # a list of dictionary objects

clean_lines = [line.strip() for line in lines]                      # removing the leading and trailing spaces
non_empty_lines = [line for line in clean_lines if line!= '']       # removing any empty lines

for line_no in non_empty_lines:
    original = []                                                   # creating arrays to store respective 
    corrected = []                                                  # data
    indexes = []                                                    
    line_token = word_tokenize(line_no)                             # splitting each word of a line into tokens
    for line_value in line_token:
        if("|" in line_value):                                      # if token contains '|' , find the respective
            indexes.append(line_token.index(line_value))            # index and append it to index[].
            w_c = line_value.split("|")                             # split the token as <original>|<corrected>
            original.append(w_c[0])                                 # and append respectively to original[] and
            corrected.append(w_c[1])                                # corrected[]
        else:
            original.append(line_value)                             # otherwise, add token to both original[] and
            corrected.append(line_value)                            # corrected[]

    # for each line, create a dictionary 
    dict_value = {'original':original,'corrected':corrected,'indexes':indexes}
    data.append(dict_value)                                         # append the dictionary to data[]                                        
    
#data[36]

assert(data[2] == {
   'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
   'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
   'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [3]:
test = data[:100]
train = data[100:]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [4]:
from collections import Counter

corrected_word_list = []                        # list of all tokens present in corrected training sentences

# collect all the corrected token(lower case) from training into 1 list
for train_line_no in range(len(train)):         
    corrected_word_list.append([train_word.lower() for train_word in train[train_line_no]['corrected']])
corrected_word_list = [item for sublist in corrected_word_list for item in sublist]

def unigram(word):
    freq_words = Counter(corrected_word_list)           # create a dictionary with:  
    return (freq_words[word])                           # key = token and value = frequency of occurance

def prob(word):
    return(unigram(word)/len(corrected_word_list))      # find probability of a token : freq(token)/total no of token

#Test your code with the following
assert(unigram("me")==87)

## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [5]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance('hello','hi'))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [6]:
all_train_word_list = list(set(corrected_word_list)) # all unique tokens from complete list of **correct** words
                                                     # not using original words, because it will give distance of 0
                                                     # and if there are large no. of those wrong words in training set 
                                                     # the probability of those words will increase, resulting in wrong
                                                     # word to be selected.
def get_candidates(token):
    dist = dict()
        
    for set_element in all_train_word_list:                     # creating a dictionary with key = unique token     
        dist[set_element] = edit_distance(token,set_element)    # value = distance b/w unique token and argument token
    
    res_word = [word for word,distance in dist.items() if distance == min(dist.values())]    
    return(sorted(res_word,reverse=True))                       # returning a list of tokens having least distance
    
#print("." in all_train_word_list)
    
# Test your code as follows
assert get_candidates("minde") == ['mine', 'mind']

## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [7]:
def correct(sentence):    
    op_sentence = []                                   # list containing the corrected output of sentence
    
    for sentence_word in sentence:                           
        if sentence_word in all_train_word_list:       # if token is present in training set list
            op_sentence.append(sentence_word)          # append it to output list
        else:                                           
            candidate_dist_prob = dict()                                         # else create a dictionary with  
            for candidate_element in get_candidates(sentence_word):              # key = candidate token, 
                candidate_dist_prob[candidate_element] = prob(candidate_element) # value = probability of occurance
            
            for candidate,probb in candidate_dist_prob.items():
                if probb == max([x for x in candidate_dist_prob.values()]):      # select candidate with max 
                    op_sentence.append(candidate)                                # probability, append to output list

    return(op_sentence)                                                          # return the output list

assert(correct(["this","whitr","cat"]) == ['this','white','cat'])     

## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [8]:
def accuracy(test):
    test_sentence_list_all = []                   # list of tokens from original sentences of test
    accuracy = 0.0
    
    # collect all the original sentences(lower case) from test into 1 list
    for test_line_no in range(len(test)):
        test_sentence_list_all.append([ele.lower() for ele in test[test_line_no]['original']])

    # for each sentence in list, find the total number of matching tokens with the list returned by correct() 
    # calculate the accuracy 
    for sentence_no in test_sentence_list_all:   
        match_case = len([i for i, j in zip(sentence_no, correct(sentence_no)) if i == j])
        accuracy += float(match_case/len(sentence_no))
    
    return((accuracy/len(test_sentence_list_all))*100)

print(accuracy(test),"%")

79.40402356176448 %


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


### Using Bigrams instead of unigrams and calculating PMI 

By using **Bigrams model** we can understand the context information of the token. It calculates the conditional probability of a token given the preceding token. By calculating bigram frequency we can determine better which token is preceeded by which word most of the times. Also, by using **PMI** we can strengthen our findings. PMI tells us which pair of bigram has high co-relation measure and provides a better understanding.

**steps taken to improve the algorithm**

1) create a list with all the corrected training tokens after removing the punctuation. 

2) create a dictionary with key = bigrams and value = PMI value.

In [9]:
new_train_word_list = [item for item in all_train_word_list if item not in string.punctuation]

PMI_bigrams = dict()
Bigram_measure_PMI = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(new_train_word_list)
for bigram, pmi in finder.score_ngrams(Bigram_measure_PMI.pmi):
    PMI_bigrams[bigram] = pmi

#PMI_bigrams

3) create a new list having stem of tokens. **(stemming)**

This is done in order to check wether the word's stem is present in list or not.

**takes a little while to execute (~5 mins)**

In [10]:
porter_stemmer = PorterStemmer()

# all_train_word_list = list(set(all_train_word_list))
for set_element in new_train_word_list:
    all_train_word_list_stem = [porter_stemmer.stem(x) for x in new_train_word_list]


#all_train_word_list_stem

4) we check each word in sentence with the corrected training tokens.

     4.1) If the word is present: we add that to our output 

     4.2) else: we check wether the stem of word is present in the list of stems.
    
         4.2.1) if present, we add the word in output. 
    
         4.2.2) else:
                * we find candidates for the word.
                * create a sequence (previous word, candidate_word)
                * find the PMI value for sequence
                * store the results in dictionary where, key = candidate and value = PMI measure
                * select the candidate with highest PMI measure
                * add the candidate to output

**why are we checking the word in list of stem of words ?**

If the list has a matching stem, we don't change the word (it means the word is not wrong, just that our training list  dosent have the exact match of the word)

else, we find the candidate of the wrong_word and replace it with word having highest PMI based on sequence formed.

In [11]:
def correct_new(sentence):    
    op_sentence = []                          # output list having corrected words
    
    for sentence_word_index in range(len(sentence)):                       # check 'word' is present in training
        if sentence[sentence_word_index] in new_train_word_list:           # list. if present, append it to 
            op_sentence.append(sentence[sentence_word_index])              # output list
        else:
            if (porter_stemmer.stem(sentence[sentence_word_index]) in all_train_word_list_stem):
                op_sentence.append(sentence[sentence_word_index])          # if not, check if stem of word is present
                                                                           # if present, append it to output list
            else:
                candidate_dist_prob = dict()                               
                for candidate_element in get_candidates(sentence[sentence_word_index]):
                    sequence = (sentence[sentence_word_index-1],candidate_element)  # sequence(previous word,candidate)
                    try:
                        candidate_dist_prob[candidate_element] = PMI_bigrams[sequence] # find the PMI measure
                    except:
                        candidate_dist_prob[candidate_element] = 0
                               
                for candidate,probb in candidate_dist_prob.items():                     # select candidate having 
                    if probb == max([x for x in candidate_dist_prob.values()]):         # highest PMI measure
                        op_sentence.append(candidate)
                        break
                    
    return(op_sentence)

## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [12]:
def accuracy_new(test):
    test_sentence_list_all = []
    accuracy = 0.0

    # collect all the original sentences(lower case) from test(removing punctuations) into 1 list
    for test_line_no in range(len(test)):
        test_sentence_list = [ele.lower() for ele in test[test_line_no]['original']if ele not in string.punctuation]
        if len(test_sentence_list)== 0:
            continue
        test_sentence_list_all.append(test_sentence_list)
    
    # for each sentence in list, find the total number of matching tokens with the list returned by correct_new() 
    # calculate the accuracy 
    for sentence_no in test_sentence_list_all:
    
        match_case = len([i for i, j in zip(sentence_no, correct_new(sentence_no)) if i == j])
        accuracy += float(match_case/len(sentence_no))
    
    return((accuracy/len(test_sentence_list_all))*100)

print(accuracy_new(test),"%")

84.6912181290406 %
